In [24]:
import requests
import pandas as pd
import json
from dateutil import parser

In [25]:
ACCOUNT_ID = "101-001-17960231-001"
API_KEY = "cdc82e28b48f1bd14d1ace5c80115808-0c51aeaf25aa2ffde07b57b4bf83f5e0"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [26]:
session = requests.Session()

In [27]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [28]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [29]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [30]:
response = session.get(url, params=None, data=None, headers=None)

In [31]:
response.status_code

200

In [32]:
data = response.json()

In [33]:
instruments_list =data['instruments']

In [34]:
len(instruments_list)

68

In [35]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [36]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [37]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k:i[k] for k in key_i}

In [38]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [39]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [41]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url=f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data=[]
        else:
            data= data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data)==0:
        return pd.DataFrame()    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']    
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']

        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])

        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code !=200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} canles, {candles_df.time.min()} {candles_df.time.max()}")

In [42]:
code, data = fetch_candles("EUR_USD", count=10, granularity='H4')
candles_df = get_candles_df(data)

In [19]:
code

200

In [45]:
# generating pairs of currencies
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [48]:
# save the data in pkl format
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4001 canles, 2022-04-27 23:00:00+00:00 2022-12-16 21:00:00+00:00
EUR_USD H4 4001 canles, 2020-05-26 01:00:00+00:00 2022-12-16 18:00:00+00:00
EUR_GBP H1 4001 canles, 2022-04-27 21:00:00+00:00 2022-12-16 21:00:00+00:00
EUR_GBP H4 4001 canles, 2020-05-26 01:00:00+00:00 2022-12-16 18:00:00+00:00
EUR_JPY H1 4001 canles, 2022-04-27 15:00:00+00:00 2022-12-16 21:00:00+00:00
EUR_JPY H4 4001 canles, 2020-05-25 17:00:00+00:00 2022-12-16 18:00:00+00:00
EUR_CHF H1 4001 canles, 2022-04-28 00:00:00+00:00 2022-12-16 21:00:00+00:00
EUR_CHF H4 4001 canles, 2020-05-26 01:00:00+00:00 2022-12-16 18:00:00+00:00
EUR_NZD H1 4001 canles, 2022-04-27 23:00:00+00:00 2022-12-16 21:00:00+00:00
EUR_NZD H4 4001 canles, 2020-05-26 01:00:00+00:00 2022-12-16 18:00:00+00:00
EUR_CAD H1 4001 canles, 2022-04-28 00:00:00+00:00 2022-12-16 21:00:00+00:00
EUR_CAD H4 4001 canles, 2020-05-26 01:00:00+00:00 2022-12-16 18:00:00+00:00
EUR_AUD H1 4001 canles, 2022-04-27 22:00:00+00:00 2022-12-16 21:00:00+00:00
EUR_AUD H4 4